In [1]:
# import env
# # Make HTTP requests
# import requests
# # Scrape data from an HTML document
# from bs4 import BeautifulSoup
# I/O
#import os
# Search and manipulate strings
import re
#import lyricsgenius
import pandas as pd
import nltk
import draft_prepare as prep
import draft_model as model
import contractions
from collections import Counter



In [2]:
# used to download lexicon data
# nltk.download('vader_lexicon')

In [3]:
# playing with genius API

# genius = lyricsgenius.Genius(env.token)

# artist = genius.search_artist("Andy Shauf", max_songs=3, sort="title")
# print(artist.songs)
# song = artist.songs[0]
# print(song.lyrics)
# df = pd.read_csv("charts.csv")
# df.head()
# df.shape
# #convert date to datetime
# df['date'] = pd.to_datetime(df['date'])
# df.head()
# first_date = pd.DataFrame(df.groupby(['artist', 'song']).date.min())
# # add a column for year and decade
# first_date['year'] = first_date.date.dt.year
# first_date['decade'] = first_date.date.dt.year - (first_date.date.dt.year % 10)
# first_date.reset_index(inplace=True)
# first_date.head()
# first_date.shape

In [4]:
# def prep_data(df):
#     '''
#     get unique artist song combos
#     make column for year and decade
#     change song to title
#     '''
#     # convert date to datetime
#     df['date'] = pd.to_datetime(df['date'])
#     # rename song column to title
#     df.rename(columns={'song': 'title'}, inplace=True)
#     # subset data to make only unique artist title combos while keeping earliest date
#     df = pd.DataFrame(df.groupby(['artist', 'title']).date.min())
#     # make new column for year and decade
#     df['year'] = df.date.dt.year
#     df['decade'] = df.date.dt.year - (df.date.dt.year % 10)
#     # reset index
#     df.reset_index(inplace=True)
#     return df
# df = prep_data(pd.read_csv("charts.csv"))
# df.head()
# df.shape

In [5]:
# This was used to get initial data


# api = lyricsgenius.Genius(env.token, verbose=False)

# def prep_songs():
#     songs = pd.read_csv('charts.csv')
#     songs['date'] = pd.to_datetime(songs.date)
#     songs = songs.rename(columns={'song': 'title'})
#     songs = pd.DataFrame(songs.groupby(['title', 'artist']).min().date).reset_index()
#     return songs

# def get_lyrics(title, artist):
#     song = api.search_song(title, artist)
#     lyrics = song.lyrics.replace('\n', ' ')
#     # TODO handle verse and chorus tags
#     return lyrics

# songs = prep_songs()

# for i in range(len(songs)-1, 0 , -1):
    
#     title = songs.iloc[[i]].title.values[0]
#     artist = songs.iloc[[i]].artist.values[0]
    
#     try:
#         songs.loc[i, 'lyrics'] = get_lyrics(title, artist)
#         songs.loc[i, 'status'] = 'lyrics acquired'
#     except: 
#         songs.loc[i, 'status'] = 'an error ocurred'
#     # save every 100 songs
#     if i % 100 == 0:
#         songs.to_csv('songs.csv')
    
#     print(f'{round((i / len(songs) * 100), 2)}%\t complete', end='\r')
#songs.to_csv('songs.csv')

# Prep data

In [6]:
songs = pd.read_csv('songs_0526.csv', index_col=0)
#songs.reset_index(drop = True, inplace=True)
#songs.drop(columns=['Unnamed: 0'], inplace=True)
songs.head()

,title,artist,date,lyrics
2,#1,Nelly,2001-10-20,#1 LyricsUh uh uh I just gotta bring it to the...
4,#9 Dream,John Lennon,1974-12-21,#9 Dream Lyrics[Verse 1] So long ago Was it in...
5,#Beautiful,Mariah Carey Featuring Miguel,2013-05-25,"#Beautiful Lyrics[Intro: Mariah Carey] Ah, ah,..."
6,#SELFIE,The Chainsmokers,2014-03-15,#SELFIE Lyrics[Verse 1] When Jason was at the ...
7,#thatPOWER,will.i.am Featuring Justin Bieber,2013-04-06,#thatPOWER Lyrics[Instrumental break] [Pre-Ch...


In [71]:
songs.isnull().sum()

title                 0
artist                0
date                  0
lyrics                0
place_words           0
unique_place_words    0
dtype: int64

In [8]:
(
songs.shape[0],
#how many lyrics contain 'Lyrics'
songs.lyrics.str.contains('Lyrics').sum(),
# how many lyrics contain 'Embed'
songs.lyrics.str.contains('Embed').sum()
)

(23762, 23762, 23762)

In [9]:
# test contractions package
contractions.fix('I\'m')


'I am'

In [10]:
songs = pd.read_csv('songs_0526.csv', index_col=0)
songs.lyrics[2] # look at raw lyrics

'#1 LyricsUh uh uh I just gotta bring it to they attention dirty, that\'s all..  You better watch who you talkin bout; runnin your mouth, like you know me You gon\' fuck around and show why the "Show Me" get called the "Show Me" Why one-on-one you can\'t hold me if your last name was Hanes Only way you wear me out is stitch my name on your pants No resident of France; but you swear I\'m from Paris Hundred-six karats - total? Naw that\'s per wrist Trying to compurr this - my chain to yo\' chain I\'m like Sprint or Motorola - no service, out of your range You out of your brains, thinkin I\'mma shout out your name You gotta come up with better ways than that to catch your fame All that pressure you applyin it\'s time to ease off Before I hit you from the blindside takin your sleeves off As much as we\'s floss, still hard to please boss Don\'t be lyin bitchin and cryin - suck it up as a loss Cause your, acts is wack, your whole label is wack And matter fact, eh eh-eh eh a-hold that [Hook] 

In [1]:
# testing for def model_clean in prepare.py

# # remove everything before and including 'lyrics'
# songs.lyrics = songs.lyrics.apply(lambda x: x.split('Lyrics')[1])
# # remove everything after and including 'Embed'
# songs.lyrics = songs.lyrics.apply(lambda x: x.rsplit('Embed')[0])
# # remove everything contained in []
# songs.lyrics = songs.lyrics.apply(lambda x: re.sub(r'\[.*?\]', '', x))
# # remove everything contained in ()
# songs.lyrics = songs.lyrics.apply(lambda x: re.sub(r'\(.*?\)', '', x))
# # expand contractions
# songs.lyrics = songs.lyrics.apply(lambda x: contractions.fix(x))

# # convert date to datetime
# songs['date'] = pd.to_datetime(songs['date'])
# # make new column for decade
# songs['decade'] = songs.date.dt.year - (songs.date.dt.year % 10)

# # songs = songs[(songs.decade != 1950) & (songs.decade != 2020)]
# songs['clean_lyrics'] = songs.lyrics.apply(prep.basic_clean)
# songs['lemmatized'] = songs.clean_lyrics.apply(prep.lemmatize)

In [12]:
songs = prep.model_clean(songs)
songs.head()

,title,artist,date,lyrics,decade
2,#1,Nelly,2001-10-20,uh uh uh got bring attention dirty better watc...,2000
4,#9 Dream,John Lennon,1974-12-21,long ago dream dream know yes know seemed real...,1970
5,#Beautiful,Mariah Carey Featuring Miguel,2013-05-25,ah ah beautiful ah ah beautiful hop back bike ...,2010
6,#SELFIE,The Chainsmokers,2014-03-15,jason table kept seeing look girl think make j...,2010
7,#thatPOWER,will.i.am Featuring Justin Bieber,2013-04-06,oh alive alive alive oh fly fly fly oh alive a...,2010


In [13]:
songs.lyrics[2]

'uh uh uh got bring attention dirty better watch talkin bout runnin mouth like know gon fuck around show show get called show oneonone cannot hold last name hanes way wear stitch name pant resident france swear paris hundredsix karat total naw per wrist trying compurr chain yo chain like sprint motorola service range brain thinkin imma shout name got come better way catch fame pressure applyin time ease hit blindside takin sleeve much wes floss still hard please bos lyin bitchin cryin suck loss act wack whole label wack matter fact eh eheh eh ahold matter like take sit write number one hey hey hey hey hey hey let ask man take winner three nobody remembers take hey hey hey hey like shake ya daddy move around let get peep touch ground hell yeah love girl willin learn willin get driver seat willin turn concerned say say say think said squash probably got ebay internet access website chat line mad got mine wind flat line ohh uncle could see could see many rapper want straight emulatin styl

# modeling tryouts

In [14]:
results = model.run_logistic_reg_models(songs)

In [15]:
results.head(3)

,model_number,model_type,train_accuracy,validate_accuracy,test_accuracy,better_than_baseline,baseline_accuracy,solver,C
0,baseline,baseline,0.205677,0.20559,0.205763,False,0.20559,NaN,NaN
1,106,LogisticRegression,0.207043,0.20634,0.206439,True,0.20559,liblinear,0.001
2,107,LogisticRegression,0.280373,0.27462,0.278928,True,0.20559,liblinear,0.01


In [16]:
results.sort_values('validate_accuracy', ascending=False).head(3)

,model_number,model_type,train_accuracy,validate_accuracy,test_accuracy,better_than_baseline,baseline_accuracy,solver,C
11,116,LogisticRegression,0.69832,0.457325,0.448897,True,0.20559,lbfgs,1
18,123,LogisticRegression,0.699043,0.457137,0.449797,True,0.20559,newton-cg,1
25,130,LogisticRegression,0.698963,0.457137,0.449797,True,0.20559,sag,1


In [17]:
results2 = model.run_decision_tree_models(songs)

In [18]:
results2.sort_values('validate_accuracy', ascending=False).head(3)

,model_number,model_type,train_accuracy,validate_accuracy,test_accuracy,better_than_baseline,max_depth,baseline_accuracy
6,6,DecisionTreeClassifier,0.411755,0.312324,0.301216,True,10,0.20559
7,7,DecisionTreeClassifier,0.999759,0.300694,0.298514,True,100,0.20559
5,5,DecisionTreeClassifier,0.321058,0.295629,0.296938,True,5,0.20559


In [19]:
results3 = model.run_random_forest_models(songs)

In [20]:
results3.sort_values('validate_accuracy', ascending=False).head(3)

,model_number,model_type,train_accuracy,validate_accuracy,test_accuracy,better_than_baseline,max_depth,min_samples_leaf,criterion,baseline_accuracy
87,94,RandomForest,0.936319,0.404802,0.403872,True,100,2,gini,0.20559
88,95,RandomForest,0.923776,0.399362,0.401846,True,100,2,entropy,0.20559
90,97,RandomForest,0.8215,0.391296,0.402972,True,100,3,entropy,0.20559


# add features
~~Replace Contractions~~

~~Word Count~~

~~Unique words per song~~

    Unique words per decade

~~Ratio verse to chorus~~

~~Number of verses~~

    Length of verses
    Length of chorus
    Sexual Word Count
    Vulgar Word Count

~~Sentiment~~

~~Topic Modeling~~

~~Number of verses~~ 

~~Number of choruses~~ 

~~Ratio of verses to choruses~~

~~Data/Time~~

~~year~~


In [23]:
songs = pd.read_csv('songs_0526.csv', index_col=0)
songs.head(3)


,title,artist,date,lyrics
2,#1,Nelly,2001-10-20,#1 LyricsUh uh uh I just gotta bring it to the...
4,#9 Dream,John Lennon,1974-12-21,#9 Dream Lyrics[Verse 1] So long ago Was it in...
5,#Beautiful,Mariah Carey Featuring Miguel,2013-05-25,"#Beautiful Lyrics[Intro: Mariah Carey] Ah, ah,..."


In [105]:
# get all words in [] from lyrics
songs['place_words'] = songs.lyrics.apply(lambda x: re.findall(r'\[.*?\]', x))
# make place_words all lowercase
songs['place_words'] = songs.place_words.apply(lambda x: [word.lower() for word in x])
# make place_words a string
songs['place_words'] = songs.place_words.apply(lambda x: ' '.join(x))



In [106]:
songs.place_words

0                                     [hook] [hook] [hook]
1        [verse 1] [pre-chorus 1] [chorus] [verse 2] [p...
2        [intro: mariah carey] [verse 1: miguel] [choru...
3        [verse 1] [drop 1] [verse 2] [drop 2] [verse 3...
4        [instrumental break] [pre-chorus: justin biebe...
                               ...                        
23757                                                     
23758                                                     
23759    [verse 1: chad kroeger] [pre-chorus: chad kroe...
23760    [intro: will smith & (sisqo)] [verse 1: will s...
23761                                                     
Name: place_words, Length: 23762, dtype: object

In [66]:
# convert from list to string
#songs.place_words = songs.place_words.apply(lambda x: ' '.join(x))

In [81]:
songs.reset_index(inplace=True)
songs.rename(columns={'index': 'old_index'}, inplace=True)

In [113]:
#make one list from column place_words
all_place_words= ''
for i in range(len(songs)):
    all_place_words += songs.place_words[i]
# what are the counts for the common words
#all_place_words
Counter(all_place_words.split()).most_common(25)



[('[verse', 27447),
 ('[chorus]', 21833),
 ('1]', 11316),
 ('2]', 11093),
 ('[chorus:', 8000),
 ('&', 7598),
 ('[bridge]', 5635),
 ('[pre-chorus]', 5020),
 ('3]', 3836),
 ('2:', 3543),
 ('1:', 3541),
 ('[outro][verse', 2688),
 ('[chorus', 2019),
 ('[chorus][verse', 1815),
 ('3:', 1737),
 ('[outro:', 1648),
 ('[pre-chorus:', 1589),
 ('[bridge:', 1481),
 ('lil', 1220),
 ('[post-chorus]', 1192),
 ('[hook]', 1130),
 ('[outro][intro]', 958),
 ('the', 942),
 ('[hook:', 935),
 ('[refrain]', 879)]

In [115]:
# How many times is chorus in place_words?
songs.place_words.str.contains('chorus').sum()


12671

In [117]:
songs.place_words[1]

'[verse 1] [pre-chorus 1] [chorus] [verse 2] [pre-chorus 2] [chorus] [outro]'

In [119]:
test_string = songs.place_words[1] 
# precleaned
#'[verse 1] [pre-chorus 1] [chorus] [verse 2] [pre-chorus 2] [chorus] [outro]'

# how many times is chorus in test_string?
test_string.count('[chorus') # this gets multiple types of chorus without pre-chorus




2

In [120]:
test_string.count('verse')

2

In [127]:
# add column for chorus count
songs['chorus_count'] = songs.place_words.apply(lambda x: x.count('[chorus'))
# add column for verse count
songs['verse_count'] = songs.place_words.apply(lambda x: x.count('verse'))
# add column for ratio of verses to chorus
songs['verse_chorus_ratio'] = songs.verse_count / songs.chorus_count
# add column for pre-chorus count
songs['pre_chorus_count'] = songs.place_words.apply(lambda x: x.count('[pre-chorus'))
# add column for outro count
songs['outro_count'] = songs.place_words.apply(lambda x: x.count('outro'))
# add column for bridge count
songs['bridge_count'] = songs.place_words.apply(lambda x: x.count('bridge'))
# add column for hook count
songs['hook_count'] = songs.place_words.apply(lambda x: x.count('hook'))
# fill NaN in with 0
songs.verse_chorus_ratio.fillna(0, inplace=True)

In [126]:
songs.sample(3)

,level_0,old_index,title,artist,date,lyrics,place_words,unique_place_words,chorus_count,verse_count,verse_chorus_ratio,pre_chorus_count,outro_count,bridge_count,hook_count
3073,3073,3658,Cecilia,Simon & Garfunkel,1970-04-11,"Cecilia Lyrics[Chorus] 'Cilia, you're breaking...",[chorus] [verse] [chorus] [bridge] [outro],"[[Outro], [Bridge], [Chorus], [Verse]]",2,1,0.500000,0,1,1,0
17982,17982,21195,Stain,A Boogie Wit da Hoodie Featuring DaBaby,2020-02-29,Stain Lyrics[Intro: A Boogie Wit da Hoodie] Po...,[intro: a boogie wit da hoodie] [chorus: a boo...,"[[Chorus: A Boogie Wit da Hoodie], [Intro: A B...",3,2,0.666667,0,0,0,0
16816,16816,19818,"Sexy, Sexy, Sexy",James Brown,1973-08-18,"Sexy, Sexy, Sexy LyricsHey, yeah She's sexy se...",,[],0,0,NaN,0,0,0,0


In [2]:
df = prep.clean_df(pd.read_csv('songs_0526.csv', index_col=0))
df.head(3)

,title,artist,date,lyrics,raw_lyrics,decade
2,#1,Nelly,2001-10-20,uh uh uh got bring attention dirty better watc...,#1 LyricsUh uh uh I just gotta bring it to the...,2000
4,#9 Dream,John Lennon,1974-12-21,long ago dream dream know yes know seemed real...,#9 Dream Lyrics[Verse 1] So long ago Was it in...,1970
5,#Beautiful,Mariah Carey Featuring Miguel,2013-05-25,ah ah beautiful ah ah beautiful hop back bike ...,"#Beautiful Lyrics[Intro: Mariah Carey] Ah, ah,...",2010


In [3]:
df = prep.add_features(df)
df.head(3)

,title,artist,date,lyrics,raw_lyrics,decade,character_count,word_count,sentiment,place_words,chorus_count,verse_count,verse_chorus_ratio,pre_chorus_count,outro_count,bridge_count,hook_count
2,#1,Nelly,2001-10-20,uh uh uh got bring attention dirty better watc...,#1 LyricsUh uh uh I just gotta bring it to the...,2000,2014,363,0.9915,[hook] [hook] [hook],0,0,0.0,0,0,0,3
4,#9 Dream,John Lennon,1974-12-21,long ago dream dream know yes know seemed real...,#9 Dream Lyrics[Verse 1] So long ago Was it in...,1970,842,135,0.9169,[verse 1] [pre-chorus 1] [chorus] [verse 2] [p...,2,2,1.0,2,1,0,0
5,#Beautiful,Mariah Carey Featuring Miguel,2013-05-25,ah ah beautiful ah ah beautiful hop back bike ...,"#Beautiful Lyrics[Intro: Mariah Carey] Ah, ah,...",2010,768,129,0.9989,[intro: mariah carey] [verse 1: miguel] [choru...,2,2,1.0,0,1,0,0


In [15]:
def get_ngrams(df):
    """
    get bigrams/trigrams for each song
    """
    # create bigram column
    print("creating bigram column", end="\r")
    df["bigrams"] = df.lyrics.apply(lambda x: list(nltk.ngrams(x.split(), 2)))
    # create bigram column
    print("creating trigram column", end="\r")
    df["trigrams"] = df.lyrics.apply(lambda x: list(nltk.ngrams(x.split(), 3)))
    return df

In [16]:
df = get_ngrams(df)
df.head(3)

,title,artist,date,lyrics,raw_lyrics,decade,character_count,word_count,sentiment,place_words,...,verse_count,verse_chorus_ratio,pre_chorus_count,outro_count,bridge_count,hook_count,bigrams,trigrams,unique_words,n_unique_words
2,#1,Nelly,2001-10-20,uh uh uh got bring attention dirty better watc...,#1 LyricsUh uh uh I just gotta bring it to the...,2000,2014,363,0.9915,[hook] [hook] [hook],...,0,0.0,0,0,0,3,"[(uh, uh), (uh, uh), (uh, got), (got, bring), ...","[(uh, uh, uh), (uh, uh, got), (uh, got, bring)...",baby sprint love number chat show man lyin dad...,228
4,#9 Dream,John Lennon,1974-12-21,long ago dream dream know yes know seemed real...,#9 Dream Lyrics[Verse 1] So long ago Was it in...,1970,842,135,0.9169,[verse 1] [pre-chorus 1] [chorus] [verse 2] [p...,...,2,1.0,2,1,0,0,"[(long, ago), (ago, dream), (dream, dream), (d...","[(long, ago, dream), (ago, dream, dream), (dre...",warm touching magic somebody away know yes riv...,49
5,#Beautiful,Mariah Carey Featuring Miguel,2013-05-25,ah ah beautiful ah ah beautiful hop back bike ...,"#Beautiful Lyrics[Intro: Mariah Carey] Ah, ah,...",2010,768,129,0.9989,[intro: mariah carey] [verse 1: miguel] [choru...,...,2,1.0,0,1,0,0,"[(ah, ah), (ah, beautiful), (beautiful, ah), (...","[(ah, ah, beautiful), (ah, beautiful, ah), (be...",as pretend baby yeah til kiss bright bike midd...,54


In [9]:
def get_unique_words(text):
    """
    Get unique words in text
    """

    # Create a list of words separated by a space
    words = text.split()
    # Create a variable to hold a list of unique words
    unique_words = set(words)
    # Join unique words list with a new string
    new_string = " ".join(unique_words)
    return new_string

In [12]:
df['unique_words'] = df.lyrics.apply(get_unique_words)
df['n_unique_words'] = df.unique_words.apply(lambda x: len(x.split()))
df.head(3)

,title,artist,date,lyrics,raw_lyrics,decade,character_count,word_count,sentiment,place_words,...,verse_count,verse_chorus_ratio,pre_chorus_count,outro_count,bridge_count,hook_count,bigrams,trigrams,unique_words,n_unique_words
2,#1,Nelly,2001-10-20,uh uh uh got bring attention dirty better watc...,#1 LyricsUh uh uh I just gotta bring it to the...,2000,2014,363,0.9915,[hook] [hook] [hook],...,0,0.0,0,0,0,3,"[(uh, uh), (uh, uh), (uh, got), (got, bring), ...","[(uh, uh, uh), (uh, uh, got), (uh, got, bring)...",baby sprint love number chat show man lyin dad...,228
4,#9 Dream,John Lennon,1974-12-21,long ago dream dream know yes know seemed real...,#9 Dream Lyrics[Verse 1] So long ago Was it in...,1970,842,135,0.9169,[verse 1] [pre-chorus 1] [chorus] [verse 2] [p...,...,2,1.0,2,1,0,0,"[(long, ago), (ago, dream), (dream, dream), (d...","[(long, ago, dream), (ago, dream, dream), (dre...",warm touching magic somebody away know yes riv...,49
5,#Beautiful,Mariah Carey Featuring Miguel,2013-05-25,ah ah beautiful ah ah beautiful hop back bike ...,"#Beautiful Lyrics[Intro: Mariah Carey] Ah, ah,...",2010,768,129,0.9989,[intro: mariah carey] [verse 1: miguel] [choru...,...,2,1.0,0,1,0,0,"[(ah, ah), (ah, beautiful), (beautiful, ah), (...","[(ah, ah, beautiful), (ah, beautiful, ah), (be...",as pretend baby yeah til kiss bright bike midd...,54


In [2]:
df= prep.get_data()

In [8]:
pd.set_option('max_columns', 25)
df.head(1)


,title,artist,date,lyrics,raw_lyrics,decade,character_count,word_count,unique_words,unique_words_count,sentiment,sentiment_category,place_words,chorus_count,verse_count,verse_chorus_ratio,pre_chorus_count,outro_count,bridge_count,hook_count,bigrams,trigrams
0,#1,Nelly,2001-10-20,uh uh uh got bring attention dirty better watc...,#1 LyricsUh uh uh I just gotta bring it to the...,2000,2014,363,touch top attention wait cricket lock eheh mea...,228,0.9915,very positive,[hook] [hook] [hook],0,0,0.0,0,0,0,3,"[(uh, uh), (uh, uh), (uh, got), (got, bring), ...","[(uh, uh, uh), (uh, uh, got), (uh, got, bring)..."
